In [78]:
from imutils import paths, face_utils
import numpy as np
import cv2
import matplotlib.pyplot as plt
import dlib
from tqdm import tqdm
import pandas as pd
import os

In [67]:
# load dlib face detector model
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('models/model.dat')

In [68]:
dataset_path = 'dataset/'
open_eyes_path = dataset_path + 'open_eye/'
closed_eyes_path = dataset_path + 'close_eye/'

In [69]:
IMG_SIZE = (34, 26)

In [73]:
imagelist = paths.list_images(dataset_path)

In [ ]:
total = 0
open = 0
closed = 0

for image in imagelist:
    if 'open' in image:
        open += 1
    elif 'close' in image:
        closed += 1
    total += 1

print('Total: {}'.format(total))
print('Open: {}'.format(open))
print('Closed: {}'.format(closed))

In [71]:
def crop_eye(img, eye_points):
    x1, y1 = np.amin(eye_points, axis=0)
    x2, y2 = np.amax(eye_points, axis=0)
    cx, cy = (x1 + x2) / 2, (y1 + y2) / 2

    w = (x2 - x1) * 1.2
    h = w * IMG_SIZE[1] / IMG_SIZE[0]

    margin_x, margin_y = w / 2, h / 2

    min_x, min_y = int(cx - margin_x), int(cy - margin_y)
    max_x, max_y = int(cx + margin_x), int(cy + margin_y)

    eye_rect = np.rint([min_x, min_y, max_x, max_y]).astype(np.int)

    eye_img = img[eye_rect[1]:eye_rect[3], eye_rect[0]:eye_rect[2]]

    return eye_img, eye_rect

In [106]:
# get eye from images with dlib
for image in tqdm(paths.list_images(dataset_path)):
    img = cv2.imread(image)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        # get the landmarks/parts for the face in box
        landmarks = predictor(gray, face)
        shapes = face_utils.shape_to_np(landmarks)


        # get the left and right eye images
        left_eye_img, left_eye_rect = crop_eye(gray, shapes[36:42])
        right_eye_img, right_eye_rect = crop_eye(gray, shapes[42:48])

        # left_eye_img = cv2.resize(left_eye, dsize=IMG_SIZE)
        # right_eye_img = cv2.resize(right_eye, dsize=IMG_SIZE)
        right_eye_img = cv2.flip(right_eye_img, flipCode=1)


        # show the images
        # plt.subplot(1, 2, 1)
        # plt.imshow(left_eye_img, cmap='gray')
        # plt.subplot(1, 2, 2)
        # plt.imshow(right_eye_img, cmap='gray')
        # plt.show()
        # print(left_eye_img.shape)
        # print(right_eye_img.shape)

        if 'open' in image:
            cv2.imwrite(open_eyes_path + 'left_eye' + image.split('/')[-1], left_eye_img)
            cv2.imwrite(open_eyes_path + 'right_eye' + image.split('/')[-1], right_eye_img)
        elif 'close' in image:
            cv2.imwrite(closed_eyes_path + 'left_eye' + image.split('/')[-1], left_eye_img)
            cv2.imwrite(closed_eyes_path + 'right_eye' + image.split('/')[-1], right_eye_img)


0it [00:00, ?it/s]/var/folders/1t/88mj6ftx7kzcj3p0wtqxbcl40000gn/T/ipykernel_61986/1779575945.py:14: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eye_rect = np.rint([min_x, min_y, max_x, max_y]).astype(np.int)
5100it [01:25, 59.43it/s]  


In [88]:
open_eyes_list = []
for image in tqdm(paths.list_images(open_eyes_path)):
    open_eyes_list.append(image)

closed_eyes_list = []
for image in tqdm(paths.list_images(closed_eyes_path)):
    closed_eyes_list.append(image)

1960it [00:00, 303688.06it/s]
1418it [00:00, 318782.39it/s]


In [90]:
all_images = open_eyes_list + closed_eyes_list

In [93]:
from random import shuffle
shuffle(all_images)

In [155]:
df = pd.read_csv('dataset.csv')

for image in tqdm(all_images):
    img = cv2.imread(image)
    img = cv2.resize(img, dsize=IMG_SIZE)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    image_list = []
    for row in img:
        for pixel in row:
            image_list.append(pixel)
    image_arr = str(image_list)
    if 'open' in image:
        label = 'open'
    elif 'close' in image:
        label = 'close'

    # concat list to dataFrame

    df = df.append({'state': label, 'image': image_arr}, ignore_index=True)

  0%|          | 0/3378 [00:00<?, ?it/s]/var/folders/1t/88mj6ftx7kzcj3p0wtqxbcl40000gn/T/ipykernel_61986/3555983585.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'state': label, 'image': image_arr}, ignore_index=True)
/var/folders/1t/88mj6ftx7kzcj3p0wtqxbcl40000gn/T/ipykernel_61986/3555983585.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'state': label, 'image': image_arr}, ignore_index=True)
/var/folders/1t/88mj6ftx7kzcj3p0wtqxbcl40000gn/T/ipykernel_61986/3555983585.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'state': label, 'image': image_arr}, ignore_index=True)
/var/folders/1t/88mj6ftx7kzcj3p0wtqxbcl40000gn/T/ipykernel_61986/3555983585.py:19: FutureW

In [160]:
df.to_csv('dataset.csv', index=False)

In [159]:
df.shape

(6252, 2)